2022년 지역별 범죄자료에 대한 시각화.
링크 : https://kosis.kr/statHtml/statHtml.do?orgId=135&tblId=DT_13501N_A120&vw_cd=MT_ZTITLE&list_id=135_001_002_001&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE

In [75]:
import pandas as pd

# 엑셀 파일 경로 설정
file_path = "C:/Users/galax/Downloads/범죄발생_지역_2022_전처리ver.xlsx"

# 엑셀 파일을 DataFrame으로 로드
df = pd.read_excel(file_path)
# 모든 데이터를 숫자형으로 변환
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# DataFrame 확인
df.head()


,범죄별,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
0,소계,295503.0,118130.0,71386.0,91182.0,42963.0,45650.0,17776.0,6907.0,391119.0,46038.0,47324.0,67829.0,53737.0,61545.0,78953.0,111487.0,27478.0
1,절도,37697.0,13808.0,9805.0,8900.0,5247.0,6070.0,1726.0,1110.0,41573.0,4636.0,5758.0,7663.0,5652.0,6202.0,10455.0,13285.0,3133.0
2,장물,51.0,30.0,13.0,25.0,4.0,8.0,2.0,1.0,62.0,5.0,9.0,11.0,4.0,6.0,8.0,13.0,4.0
3,사기,58894.0,27903.0,15335.0,17912.0,8746.0,12188.0,2974.0,1575.0,79005.0,9430.0,10767.0,14285.0,12735.0,10265.0,16310.0,24925.0,6109.0
4,횡령,17141.0,4625.0,2702.0,4280.0,1738.0,1524.0,393.0,249.0,15801.0,1203.0,1273.0,1833.0,1264.0,1295.0,2106.0,3059.0,868.0


In [76]:
# 정규화

# df_z DataFrame에 복사
df_z = df.copy()

# 첫 번째 열을 제외하고 각 원소를 해당 열의 첫 번째 행으로 나누기
df_z.iloc[:, 1:] = df_z.iloc[:, 1:].div(df_z.iloc[0, 1:])
df_z = df_z.drop(0)

df_z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 1 to 164
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   범죄별     164 non-null    object
 1   서울      156 non-null    object
 2   부산      160 non-null    object
 3   대구      146 non-null    object
 4   인천      154 non-null    object
 5   광주      143 non-null    object
 6   대전      142 non-null    object
 7   울산      138 non-null    object
 8   세종      109 non-null    object
 9   경기      159 non-null    object
 10  강원      155 non-null    object
 11  충북      143 non-null    object
 12  충남      153 non-null    object
 13  전북      155 non-null    object
 14  전남      151 non-null    object
 15  경북      154 non-null    object
 16  경남      157 non-null    object
 17  제주      129 non-null    object
dtypes: object(18)
memory usage: 23.2+ KB


In [77]:
df_z.head()

,범죄별,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
1,절도,0.127569,0.116888,0.137352,0.097607,0.122128,0.132968,0.097097,0.160707,0.106292,0.100699,0.121672,0.112975,0.105179,0.100772,0.132421,0.119162,0.114018
2,장물,0.000173,0.000254,0.000182,0.000274,0.000093,0.000175,0.000113,0.000145,0.000159,0.000109,0.00019,0.000162,0.000074,0.000097,0.000101,0.000117,0.000146
3,사기,0.199301,0.236206,0.214818,0.196442,0.203571,0.266988,0.167304,0.22803,0.201997,0.204831,0.227517,0.210603,0.236988,0.166789,0.206579,0.223569,0.222323
4,횡령,0.058006,0.039152,0.037851,0.046939,0.040453,0.033384,0.022108,0.03605,0.040399,0.026131,0.0269,0.027024,0.023522,0.021042,0.026674,0.027438,0.031589
5,배임,0.003858,0.002633,0.002381,0.002336,0.003049,0.002957,0.003882,0.002316,0.002539,0.002324,0.001965,0.00286,0.002177,0.002307,0.002153,0.002449,0.002547


이상치 탐지

이상치를 찾는 방법 중 하나로 Z-score를 사용. Z-score는 평균에서 표준편차의 몇 배만큼 떨어져 있는지를 나타내는 값으로, 이를 이용하여 이상치를 탐지.

Z-score를 계산하여 특정 임계값 이상인 값들을 이상치로 간주. (일반적으로 Z-score가 2 이상인 값을 이상치로 간주)

아래는 각 행의 각 열에 대한 Z-score를 계산하고, 특정 임계값을 초과하는 값들을 이상치로 간주하여 이를 출력

In [79]:
# NaN 값을 0으로 대체
df_z.fillna(0, inplace=True)
# NaN 값 확인
print(df_z.isnull().sum())

범죄별    0
서울     0
부산     0
대구     0
인천     0
광주     0
대전     0
울산     0
세종     0
경기     0
강원     0
충북     0
충남     0
전북     0
전남     0
경북     0
경남     0
제주     0
dtype: int64


In [80]:
# 1열을 행 인덱스로 설정
df_z.set_index(df_z.columns[0], inplace=True)

In [83]:
def detect_outliers_zscore(df, threshold=3): #threshold는 2 or 3
    outliers = []
    for index, row in df.iterrows():
        z_scores = (row - np.mean(row)) / np.std(row)
        outlier_indices = np.where(np.abs(z_scores) > threshold)[0]
        for idx in outlier_indices:
            outliers.append((index, df.columns[idx], row[idx]))
    return outliers

outliers = detect_outliers_zscore(df_z)
print("이상치 인덱스:", outliers)

이상치 인덱스: [('살인', '광주', 6.982752601075344e-05), ('인장', '전남', 0.0036558615647087496), ('수뢰', '광주', 0.0007215511021111189), ('도박과 복표', '울산', 0.005794329432943294), ('명예', '서울', 0.039410767403376616), ('주거침입', '제주', 0.028204381687167917), ('비밀침해', '대전', 0.0003943044906900329), ('내란의죄', '세종', 0.0002895613146083683), ('음용수에 관한죄', '강원', 0.00010860593422824623), ('가정폭력 범죄의 처벌등에 관한 특례법', '제주', 0.0026566707911783974), ('건설기계관리법', '강원', 0.001238107650202007), ('고용보험법', '울산', 0.007144464446444645), ('공유재산 및 물품관리법', '부산', 0.00043172775755523574), ('공인중개사법', '세종', 0.002026929202258578), ('관세법', '인천', 0.001239279682393455), ('국가기술자격법', '광주', 0.0002094825780322603), ('국민체육진흥법', '울산', 0.0013501350135013501), ('낚시관리 및 육성법', '제주', 0.02103500982604265), ('대부업 등의 등록 및 금융이용자 보호에 관한법률', '부산', 0.002395665791924151), ('독점규제 및 공정거래에 관한법률', '서울', 9.475369116387989e-05), ('디자인보호법', '대전', 0.0005914567360350493), ('마약류 관리에 관한법률(대마)', '인천', 0.004671974731854971), ('마약류 관리에 관한법률(향정)', '인천', 0.015715821105042663), ('배

In [88]:
def detect_outliers_zscore(df, threshold=3): #threshold는 2 or 3
    outliers = []
    for index, row in df.iterrows():
        z_scores = (row - np.mean(row)) / np.std(row)
        outlier_indices = np.where(np.abs(z_scores) > threshold)[0]
        for idx in outlier_indices:
            outliers.append((index, df.columns[idx], row[idx]))
    return outliers


outliers = detect_outliers_zscore(df_z)

# 이상치를 DataFrame으로 변환
outliers_df = pd.DataFrame(outliers, columns=['행 인덱스', '열 인덱스', '값'])
outliers_df.set_index(['행 인덱스', '열 인덱스'], inplace=True)

# 행 인덱스를 기준으로 그룹화하여 출력
grouped_outliers = outliers_df.groupby(level=0).apply(lambda x: x.droplevel(0))
grouped_outliers

,,값
행 인덱스,열 인덱스,
가정폭력 범죄의 처벌등에 관한 특례법,제주,0.002657
건설기계관리법,강원,0.001238
고용보험법,울산,0.007144
공유재산 및 물품관리법,부산,0.000432
공인중개사법,세종,0.002027
관세법,인천,0.001239
국가기술자격법,광주,0.000209
국민체육진흥법,울산,0.001350
기타특별법,울산,0.130119


In [91]:
# Excel 파일로 저장
grouped_outliers.to_excel("outliers.xlsx", index=True)